# SONATA 용 IPL 엑셀 파일을 APECS 위협 라이브러리로 변환
* 버젼 : 1.0
* 날짜 : 2022-11-18

In [1]:
import xlwings as xw
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcl
import seaborn as sns

from datetime import datetime
import sqlite3

import random

# Machine learning 
from sklearn.model_selection import cross_validate
from sklearn.decomposition import PCA


pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)

plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'NanumGothic'


In [23]:
def MakeRadarTable( n_df, radarname ):
    global platform_index
    global radar_index
    
    elnot_df = n_df['EmitterNotation'].unique()
    elnot_df = np.sort( elnot_df )
    
    for index, elnot in enumerate(elnot_df):
        radar_index = radar_index + 1
        
        rows = n_df[ n_df['EmitterNotation'] == elnot ]        
        
        # 레이더 정보 저장
        elnot_rep = 'A' + '{0:03d}'.format( radar_index ) + 'A'
        #sql = f"INSERT INTO RADAR(RADAR_INDEX, ELNOT, NICK_NAME, PRIORITY, STATUS, ACCEPTED, PRIMARY_PLATFORM, DATE_LAST_UPDATED, DATE_LAST_REVIEWED, DATE_ACTIVATED, DATE_INACTIVATED, TIME_INACTIVATED, COMBINED_DATE_LAST_UPDATED ) VALUES ( {radar_index}, '{elnot}', '{radarname}_{radar_index}', {rows.iloc[0].RadarPriority}, 'ACTIVE', 1, 1, '', '', '', '', 200, '' )"
        sql = f"INSERT INTO RADAR(RADAR_INDEX, ELNOT, NICK_NAME, PRIORITY, STATUS, ACCEPTED, PRIMARY_PLATFORM, DATE_LAST_UPDATED, DATE_LAST_REVIEWED, DATE_ACTIVATED, DATE_INACTIVATED, TIME_INACTIVATED, COMBINED_DATE_LAST_UPDATED ) VALUES ( {radar_index}, '{elnot_rep}', '{radarname}_{radar_index}', {rows.iloc[0].RadarPriority}, 'ACTIVE', 1, 1, '', '', '', '', 200, '' )"
        #print( sql );
        cur.execute( sql )
        
        # 레이더 커멘트 정보 저장
        sql = f"INSERT INTO RADAR_COMMENTS (RADAR_INDEX, TITLE, DATE_CREATED, DATE_LAST_UPDATED, COMMENTS) VALUES ( 1, '{radarname} 커멘트#{radar_index}', '', '', '커멘트를 작성하면 됩니다.')"
        #print( sql )
        cur.execute( sql )
        
        # 레이더 플레폼 저장
        platform_name = rows.iloc[0][ ['Platform 1 Name', 'Platform 2 Name', 'Platform 3 Name', 'Platform 4 Name', 'Platform 5 Name'] ]
        #print( type(platform_name[0]) )
        
        for i in range(5):
            if platform_name[i] != '-' and platform_name[i] != '':
                platform_index = platform_index + 1
                
                sql = f"INSERT INTO ASSOC_PLATFORM(RADAR_INDEX, PLATFORM_INDEX) VALUES ( {index+1}, {platform_index} )"
                #print( sql )
                cur.execute( sql )
                
                sql = f"INSERT INTO PLATFORM(RADAR_INDEX, PLATFORM_INDEX, PLATFORM) VALUES ( {index+1}, {platform_index}, '{platform_name[i]}' )"
                #print( sql )
                cur.execute( sql )
                
    return index

In [24]:
def MakeRadarModeTable( n_df, radarname ):
    global radar_index
    global radarmode_index

    global rfSeq
    global rfIndex

    global priSeq
    global priIndex
    
    coRadarModeIndex = 0

    elnot_df = n_df['EmitterNotation'].unique()
    elnot_df = np.sort( elnot_df )

    for index, value in enumerate(elnot_df):
        rows = n_df[ n_df['EmitterNotation'] == value ]
        radarmode_name = rows.loc[:,['RadarMode Name']]
        
        radar_index = radar_index + 1
        for idx, row in rows.iterrows():
            coRadarModeIndex = coRadarModeIndex + 1
            radarmode_index = radarmode_index + 1
            
            # 레이더모드-사이클 정보 저장
            #sql = f"INSERT INTO RADARMODE_LIFECYCLE(RADAR_INDEX, RADARMODE_INDEX, RADARMODE_NAME ) VALUES ({radar_index}, {radarmode_index}, '{row['RadarMode Name']}' )"
            sql = f"INSERT INTO RADARMODE_LIFECYCLE(RADAR_INDEX, RADARMODE_INDEX, RADARMODE_NAME ) VALUES ({radar_index}, {radarmode_index}, '레이더명_{radarmode_index}' )"
            #print( sql )
            cur.execute( sql )
                        
            # 레이더모드 정보 저장
            sql = f"INSERT INTO RADARMODE VALUES ({radarmode_index}, '{nowStr}', '', '', '', '', {int(row['Pulse Type'])}, {row['Category']}, 0.0, 0.0, {int(row['RF Type'])}, {row['FreqLow(MHz)']}, {row['FreqHigh(MHz)']}, {row['CenterFreq(MHz)']-5}, {row['CenterFreq(MHz)']+5}, \
0, {row['SwitchLevel']}, {row['PatternType']}, {row['PatternPeriodLow(usec)']}, {row['PatternPeriodHigh(usec)']}, 0.0, {int(row['PRI Type'])}, {row['PRILow(usec)']}, {row['PRIHigh(usec)']}, {row['CenterPRI(usec)']-5}, {row['CenterPRI(usec)']+5}, 0, {row['Stagger/Switch Level']}, {row['PRI Pattern Type']}, {row['PRI PatternPeriodLow(usec)']}, {row['PRI PatternPeriodHigh(usec)']}, \
0, 0, {row['PW Low(usec)']}, {row['PW High(usec)']}, {int(row['AS Type'])}, {int(row['AS Low(usec)'])}, {int(row['AS High(usec)'])}, 0, 0.0, 0.0, 'ZZ', {row['RadarModePriority']}, 1 )"            
            #print( sql )
            cur.execute( sql )
            
            # 주파수 레벨값 저장
            bRFSequenceName = True
            for i in range( int( row['SwitchLevel'] ) ):                
                rfIndex = rfIndex + 1
                
                filterLowStr = f"Dwell Freq{i+1} Low(MHz)"
                filterHighStr = f"Dwell Freq{i+1} High(MHz)"
                sql = f"INSERT INTO RADAR_RF_VALUES (RADARMODE_INDEX, RF_MIN, RF_MAX ) VALUES ( {radarmode_index}, {row[filterLowStr]}, {row[filterHighStr]} )"
                #print( filterLowStr, filterHighStr, sql )
                cur.execute( sql )
                
                if bRFSequenceName == True:
                    bRFSequenceName = False
                    rfSeq = rfSeq + 1
                    sql = f"INSERT INTO RADAR_RF_SEQUENCE_NAME (RADARMODE_INDEX, RF_SEQ_ID, RF_SEQ_NAME ) VALUES ( {radarmode_index}, {rfSeq}, 'RF 시퀀스#{rfSeq}' )"
                    #print( sql )
                    cur.execute( sql )
                
                sql = f"INSERT INTO RADAR_RF_SEQUENCE (RADARMODE_INDEX, RF_SEQ_ID, RF_INDEX) VALUES ( {radarmode_index}, {rfSeq}, {rfIndex} )"
                #print( sql )
                cur.execute( sql )            
            
            # PRI 레벨값 저장
            bPriSequenceName = True
            for i in range( int( row['Stagger/Switch Level'] ) ):                
                priIndex = priIndex + 1
                
                filterLowStr = f"SD Value{i+1} Low(MHz)"
                filterHighStr = f"SD Value{i+1} High(MHz)"
                sql = f"INSERT INTO RADAR_PRI_VALUES (RADARMODE_INDEX, PRI_MIN, PRI_MAX, DWELL_DURATION_MIN, DWELL_DURATION_MAX, PULSES_PER_DWELL, TRANSITION_MIN, TRANSITION_MAX) VALUES ( {radarmode_index}, {row[filterLowStr]}, {row[filterHighStr]}, 0.0, 0.0, 100, 0.0, 0.0 )"
                #print( filterLowStr, filterHighStr, sql )
                cur.execute( sql )

                if bPriSequenceName == True:
                    bPriSequenceName = False
                    priSeq = priSeq + 1
                    sql = f"INSERT INTO RADAR_PRI_SEQUENCE_NAME (RADARMODE_INDEX, PRI_SEQ_ID, PRI_SEQ_NAME ) VALUES ( {radarmode_index}, {priSeq}, 'PRI 시퀀스#{priSeq}' )"
                    #print( sql )
                    cur.execute( sql )
                
                sql = f"INSERT INTO RADAR_PRI_SEQUENCE (RADARMODE_INDEX, PRI_SEQ_ID, PRI_INDEX) VALUES ( {radarmode_index}, {priSeq}, {priIndex} )"
                #print( sql )
                cur.execute( sql )
                
    return coRadarModeIndex

# 엑셀용 IPL 데이터 불러오기

In [8]:
raw_xl = xw.Book( 'TestLibrary_시험용(헤더 수정).xlsx')

sh = raw_xl.sheets(1)

# 위협 라이브로리 로딩
n_df_1 = sh.range('A1').options(pd.DataFrame, expand='table').value
n_df_1.insert( 4, 'Delta Freq(MHz)', n_df_1['FreqHigh(MHz)'] - n_df_1['FreqLow(MHz)'] )
n_df_1.insert( 4, 'CenterFreq(MHz)', ( n_df_1['FreqHigh(MHz)'] + n_df_1['FreqLow(MHz)'] ) / 2.0 )

n_df_1.insert( 13, 'Delta PRI(usec)', n_df_1['PRIHigh(usec)'] - n_df_1['PRILow(usec)'] )
n_df_1.insert( 13, 'CenterPRI(usec)', ( n_df_1['PRIHigh(usec)'] + n_df_1['PRILow(usec)'] ) / 2.0 )

n_df_1.insert( 24, 'Delta PW(usec)', n_df_1['PW High(usec)'] - n_df_1['PW Low(usec)'] )
n_df_1.insert( 24, 'CenterPW(usec)', ( n_df_1['PW High(usec)'] + n_df_1['PW Low(usec)'] ) / 2.0 )

#n_df_1.index = n_df_1.index.map( lambda x : int(x) )

In [9]:
raw_xl = xw.Book( '사용자 라이브러리.xlsx')

sh = raw_xl.sheets(1)

# 위협 라이브로리 로딩
n_df_2 = sh.range('A1').options(pd.DataFrame, expand='table').value
n_df_2.insert( 4, 'Delta Freq(MHz)', n_df_2['FreqHigh(MHz)'] - n_df_2['FreqLow(MHz)'] )
n_df_2.insert( 4, 'CenterFreq(MHz)', ( n_df_2['FreqHigh(MHz)'] + n_df_2['FreqLow(MHz)'] ) / 2.0 )

n_df_2.insert( 13, 'Delta PRI(usec)', n_df_2['PRIHigh(usec)'] - n_df_2['PRILow(usec)'] )
n_df_2.insert( 13, 'CenterPRI(usec)', ( n_df_2['PRIHigh(usec)'] + n_df_2['PRILow(usec)'] ) / 2.0 )

n_df_2.insert( 24, 'Delta PW(usec)', n_df_2['PW High(usec)'] - n_df_2['PW Low(usec)'] )
n_df_2.insert( 24, 'CenterPW(usec)', ( n_df_2['PW High(usec)'] + n_df_2['PW Low(usec)'] ) / 2.0 )

# SQLITE 연동

In [10]:
# DB 생성
#conn = sqlite3.connect( "cedeob.sqlite3", isolation_level=None )
conn = sqlite3.connect( "cedeob.sqlite3" )

# 커서 연결
cur = conn.cursor()

# 레이더/레이더모드 관련 테이블 삭제

In [25]:
now = datetime.now()
nowStr = now.strftime( '%Y-%m-%d %H:%M:%S' )

print( f"현재 시간은 {nowStr} 입니다.\n\n" )

try:
    cur.execute( 'delete from RADAR_RF_SEQUENCE_NAME;')
    cur.execute( 'delete from RADAR_RF_SEQUENCE;')
    cur.execute( 'delete from RADAR_RF_VALUES;')
    cur.execute( 'delete from RADAR_PRI_SEQUENCE_NAME;')
    cur.execute( 'delete from RADAR_PRI_SEQUENCE;')
    cur.execute( 'delete from RADAR_PRI_VALUES;')

    cur.execute( 'delete from RADARMODE;')
                  
    cur.execute( 'delete from RADAR_COMMENTS;')

    cur.execute( 'delete from RADAR_RF_SPOT_VALUES;')
    cur.execute( 'delete from RADAR_PRI_SPOT_VALUES;')

    cur.execute( 'delete from RADARMODE_LIFECYCLE;')

    cur.execute( 'delete from ASSOC_PLATFORM;')
    cur.execute( 'delete from PLATFORM;')

    cur.execute( 'delete from RADAR;')
    
    print( "===> 테이블을 정상적으로 삭제 했습니다." )
    
    cur.execute( "update sqlite_sequence set seq=0 where name='RADAR_COMMENTS';")
    cur.execute( "update sqlite_sequence set seq=0 where name='RADAR_RF_VALUES';")
    
    cur.execute( "update sqlite_sequence set seq=0 where name='RADAR_RF_SEQUENCE';")
    
    cur.execute( "update sqlite_sequence set seq=0 where name='RADAR_PRI_VALUES';")
    cur.execute( "update sqlite_sequence set seq=0 where name='RADAR_PRI_SEQUENCE';")
    
    print( "===> 자동 증가를 초기화 했습니다." )

    conn.commit()
    
except Exception as exception:
    print( exception )
    print( '에러 발생함' )    


현재 시간은 2023-09-17 14:26:19 입니다.


===> 테이블을 정상적으로 삭제 했습니다.
===> 자동 증가를 초기화 했습니다.


# 레이더 테이블 생성

In [26]:
try:
    platform_index = 0
    radar_index = 0;
    
    coStandaraLibrary = MakeRadarTable( n_df_1, '레이더' )
    coUserLibrary = MakeRadarTable( n_df_2, '사용자' )

    conn.commit()

    print( '===> 정상적으로 레이더 {}개를 변환했으며, 사용자 레이더 {}개를 추가했습니다.'.format( coStandaraLibrary, coUserLibrary ) )
        
except Exception as exception:
    print( exception )
    print( '에러 발생함' )

===> 정상적으로 레이더 278개를 변환했으며, 사용자 레이더 62개를 추가했습니다.


# 레이더모드(라이프싸이클, 레이더모드 정보) 테이블 생성

In [27]:
try:
    rfSeq = 0
    rfIndex = 0
    priSeq = 0
    priIndex = 0
    radar_index = 0
    radarmode_index = 0
    
    coStandaraLibrary = MakeRadarModeTable( n_df_1, "레이더" )
    coUserLibrary = MakeRadarModeTable( n_df_2, "사용자" )

    conn.commit()            
    print( '===> 정상적으로 레이더 모드 {}개를 변환했고, 사용자 레이더 모드 {}개를 변환했습니다.'.format( coStandaraLibrary, coUserLibrary ) )

except Exception as exception:
    print( exception )
    print( '에러 발생 !' )   
    
! copySqlite.bat    

===> 정상적으로 레이더 모드 2000개를 변환했고, 사용자 레이더 모드 68개를 변환했습니다.

(base) E:\APECS\Dev\PocketSONATA\dev_\Document\IPL2CED>echo off 
        1개 파일이 복사되었습니다.
        1개 파일이 복사되었습니다.
        1개 파일이 복사되었습니다.


 # 사용자 레이더 모드 생성

In [86]:
def GenerateRadarModeFREQ( n, user_radarmode_index, nowStr ):
    
    # FIXED 일때
    if n == 0:
        sqlString = f"INSERT INTO RADARMODE VALUES ({user_radarmode_index}, '{nowStr}', '', '', '', '', 1, 0.0, 0.0, {n}, 17490, 17510, 17500, 17500, 0, 0, 0, 0, 0, 0.0, "
       
    # HOPPING 일때    
    elif n == 1:
        sqlString = f"INSERT INTO RADARMODE VALUES ({user_radarmode_index}, '{nowStr}', '', '', '', '', 1, 0.0, 0.0, {n}, 1000, 2000, 1500, 1500, 0, 32, 0, 0, 0, 0.0, "
        
    # AGILE 일때    
    elif n == 2:
        sqlString = f"INSERT INTO RADARMODE VALUES ({user_radarmode_index}, '{nowStr}', '', '', '', '', 1, 0.0, 0.0, {n}, 500, 18000, 7500, 7500, 0, 0, 0, 0, 0, 0.0, "
        
    # PATTERN 일때
    else:
        sqlString = f"INSERT INTO RADARMODE VALUES ({user_radarmode_index}, '{nowStr}', '', '', '', '', 1, 0.0, 0.0, 3, 500, 18000, 7500, 7500, 0, 0, 0, 0, 0, 0.0, "
    
    return sqlString

In [87]:
def GenerateRadarModePRI(n, sqlString ):
    
    # STABLE PRI 일때
    if n == 0:
        sqlString += f"{n}, 100, 100, 100, 100, 0, 0, 0, 0, 0, 0, 0, 0, 200, 0, 0, 0, 0, 0.0, 0.0, 'ZZ', 100, 0, 0, 1 )"
       
    # JITTER PRI 일때    
    elif n == 1:
        sqlString += f"{n}, 450, 550, 500, 500, 0, 0, 0, 0, 0, 0, 0, 0, 200, 0, 0, 0, 0, 0.0, 0.0, 'ZZ', 100, 0, 0, 1 )"
        
    # D&S PRI 일때    
    elif n == 2:
        sqlString += f"{n}, 100, 100, 95, 105, 0, 0, 0, 0, 0, 0, 0, 0, 200, 0, 0, 0, 0, 0.0, 0.0, 'ZZ', 100, 0, 0, 1 )"        
        
    # Stagger PRI 일때    
    elif n == 3:
        sqlString += f"{n}, 100, 100, 95, 105, 0, 0, 0, 0, 0, 0, 0, 0, 200, 0, 0, 0, 0, 0.0, 0.0, 'ZZ', 100, 0, 0, 1 )"                
        
    # Pattern PRI 일때    
    elif n == 4:
        sqlString += f"{n}, 100, 100, 95, 105, 0, 0, 0, 0, 0, 0, 0, 0, 200, 0, 0, 0, 0, 0.0, 0.0, 'ZZ', 100, 0, 0, 1 )"                        
        
    else:
        sqlString += f"{n}, 100, 100, 95, 105, 0, 0, 0, 0, 0, 0, 0, 0, 200, 0, 0, 0, 0, 0.0, 0.0, 'ZZ', 100, 0, 0, 1 )"    
    
    return sqlString

In [106]:

fixed_freq = 17500

try:
    
    #cur.execute( sql )    
    
    # 사용자 레이더 모드 추기
    for i in range(1, 7 * 8 + 1 ):
        user_index = index + i
        user_radarmode_index = radarmode_index + i
        
        # 레이더 추가
        value = f"X{i:03d}A"
        sql = f"INSERT INTO RADAR(RADAR_INDEX, ELNOT, NICK_NAME, PRIORITY, STATUS, ACCEPTED, PRIMARY_PLATFORM, DATE_LAST_UPDATED, DATE_LAST_REVIEWED, TIME_INACTIVATED) VALUES ( {user_index+1}, '{value}', '레이더_{user_index+1}', 100, 'ACTIVE', 1, 1, '', '', 200 )"
        #print( sql );
        cur.execute( sql )        

        # 레이더모드-사이클 정보 저장        
        sql = f"INSERT INTO RADARMODE_LIFECYCLE(RADAR_INDEX, RADARMODE_INDEX, RADARMODE_NAME ) VALUES ({user_index+1}, {user_radarmode_index}, '사용자 레이더모드{i}' )"
        #print( sql )
        cur.execute( sql )
    
    for i in range( 7 ):
        for j in range( 1, 9 ):
            sql = GenerateRadarModeFREQ( i, radarmode_index+(i*8)+j, nowStr )
            sql = GenerateRadarModePRI( j, sql )
            #print( sql )
            cur.execute( sql )

    conn.commit()                

except Exception as exception:
    print( exception )
    print( '에러 발생 !' )

INSERT INTO RADAR(RADAR_INDEX, ELNOT, NICK_NAME, PRIORITY, STATUS, ACCEPTED, PRIMARY_PLATFORM, DATE_LAST_UPDATED, DATE_LAST_REVIEWED, TIME_INACTIVATED) VALUES ( 280, 'X001A', '레이더_280', 100, 'ACTIVE', 1, 1, '', '', 200 )
INSERT INTO RADAR(RADAR_INDEX, ELNOT, NICK_NAME, PRIORITY, STATUS, ACCEPTED, PRIMARY_PLATFORM, DATE_LAST_UPDATED, DATE_LAST_REVIEWED, TIME_INACTIVATED) VALUES ( 281, 'X002A', '레이더_281', 100, 'ACTIVE', 1, 1, '', '', 200 )
INSERT INTO RADAR(RADAR_INDEX, ELNOT, NICK_NAME, PRIORITY, STATUS, ACCEPTED, PRIMARY_PLATFORM, DATE_LAST_UPDATED, DATE_LAST_REVIEWED, TIME_INACTIVATED) VALUES ( 282, 'X003A', '레이더_282', 100, 'ACTIVE', 1, 1, '', '', 200 )
INSERT INTO RADAR(RADAR_INDEX, ELNOT, NICK_NAME, PRIORITY, STATUS, ACCEPTED, PRIMARY_PLATFORM, DATE_LAST_UPDATED, DATE_LAST_REVIEWED, TIME_INACTIVATED) VALUES ( 283, 'X004A', '레이더_283', 100, 'ACTIVE', 1, 1, '', '', 200 )
INSERT INTO RADAR(RADAR_INDEX, ELNOT, NICK_NAME, PRIORITY, STATUS, ACCEPTED, PRIMARY_PLATFORM, DATE_LAST_UPDATED, DA

In [21]:
try:
    # 사용자 레이러 모드 추기
    user_index = index + 1
    user_radarmode_index = radarmode_index + 1
    
    # 레이더모드-사이클 정보 저장        
    sql = f"INSERT INTO RADARMODE_LIFECYCLE(RADAR_INDEX, RADARMODE_INDEX, RADARMODE_NAME ) VALUES ({user_index+1}, {user_radarmode_index}, '사용자 레이더모드' )"
    print( sql )
    #cur.execute( sql )

except Exception as exception:
    print( exception )
    print( '에러 발생 !' )   

print( radarmode_index )

sql = f"INSERT INTO RADARMODE_LIFECYCLE(RADAR_INDEX, RADARMODE_INDEX, RADARMODE_NAME ) VALUES ({index+1}, {radarmode_index}, '{row['RadarMode Name']}' )"
print( sql )
#cur.execute( sql )

INSERT INTO RADARMODE_LIFECYCLE(RADAR_INDEX, RADARMODE_INDEX, RADARMODE_NAME ) VALUES (280, 2001, '사용자 레이더모드' )
2000
INSERT INTO RADARMODE_LIFECYCLE(RADAR_INDEX, RADARMODE_INDEX, RADARMODE_NAME ) VALUES (279, 2000, 'No Name' )


# 위협/장비 테이블 삭제

In [9]:
now = datetime.now()
nowStr = now.strftime( '%Y-%m-%d %H:%M:%S' )

print( f"현재 시간은 {nowStr} 입니다.\n\n" )

try:
    cur.execute( 'delete from DEVICE;' )
    cur.execute( 'delete from THREAT;')
    
    print( "===> 테이블을 정상적으로 삭제 했습니다." )
    
    
    print( "===> 자동 증가를 초기화 했습니다." )

    conn.commit()
    
except Exception as exception:
    print( exception )
    print( '에러 발생함' )   


현재 시간은 2023-07-21 09:16:53 입니다.


===> 테이블을 정상적으로 삭제 했습니다.
===> 자동 증가를 초기화 했습니다.


# 위협/장비 테이블 생성

In [10]:
def RandomFloat32( start_value, end_value ):
    return random.random() * ( end_value - start_value ) + start_value;

try:    
    device_index = 1
    for index, value in enumerate(elnot_df):
        # 위경도 랜던 모의
        latitude = RandomFloat32( 38, 39 );
        longitude = RandomFloat32( 127, 129 );
        #print( latitude, longitude )
        
        # 위협 정보 저장
        sql = f"INSERT INTO THREAT (THREAT_INDEX, THREAT_NAME, SITE_NAME, PIN, PLACE_NAME_KOR, FRIEND_OR_FOE, PRIORITY, CATEGORY, PLATFORM_TYPE, SYMBOL_CODE ) VALUES ( {index+1}, '위협명 #{index+1}', '지명#{index+1}', '{device_index}', '한국지명', 0, 100, 1, 1, '2525B 심벌' );";
        #print( sql )
        cur.execute( sql )
             
        # 장비 정보 저정
        
        sql = f"INSERT INTO DEVICE (THREAT_INDEX, DEVICE_INDEX, ELNOT, DEVICE_NAME, IDENTIFICATION_RANGE, LATITUDE, LONGITUDE, ELEVATION ) VALUES ( {index+1}, {device_index}, '{value}', '장비명 #{index+1}', 10.0, {latitude}, {longitude}, 10.0 );";
        #print( sql )      
        cur.execute( sql )
        
        device_index = device_index + 1
      
    conn.commit()

    print( '===> 정상적으로 위협/장비 {}/{}개를 생성했습니다.'.format( len(elnot_df), len(elnot_df) ) )
        
except Exception as exception:
    print( exception )
    print( '에러 발생함' )

===> 정상적으로 위협/장비 279/279개를 생성했습니다.


# CEDEOB DB 파일 복사

In [14]:
! copySqlite.bat


(base) E:\APECS\Dev\PocketSONATA\dev_\Document\IPL2CED>echo off 
        1개 파일이 복사되었습니다.
        1개 파일이 복사되었습니다.


In [12]:
conn.close()

print( "DB 를 종료합니다 !!!" )

DB 를 종료합니다 !!!
